This Notebook is where we are fine-tuning a model for comment generation. We do that by fine-tuning the model for a Masked Language Replacement task using the following dataset: https://github.com/IsraelAbebe/An-Amharic-News-Text-classification-Dataset

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers sentencepiece torch datasets --quiet

     |████████████████████████████████| 2.5MB 4.0MB/s 
     |████████████████████████████████| 1.2MB 26.4MB/s 
     |████████████████████████████████| 266kB 32.6MB/s 
     |████████████████████████████████| 901kB 32.6MB/s 
     |████████████████████████████████| 3.3MB 39.6MB/s 
     |████████████████████████████████| 122kB 40.3MB/s 
     |████████████████████████████████| 245kB 38.4MB/s 


In [ ]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-finetuned-amharic")
tokenizer = AutoTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/gdrive/MyDrive/amharic/AmharicNewsDataset.csv')
df = df.drop(['headline','category','date','views', 'link'], axis=1)

train, test = train_test_split(df, test_size=0.2)

train.to_csv("/content/gdrive/MyDrive/amharic/AmharicNewsDatasetTrunc.csv", index=False)
test.to_csv("/content/gdrive/MyDrive/amharic/AmharicNewsDatasetTruncTest.csv", index=False)

In [ ]:
df.head()

,article
0,ብርሃን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየዓመቱ የሚያዘጋጀው የክለቦች ቻ...
1,የአዲስ ዘመን ጋዜጣ ቀደምት ዘገባዎች በእጅጉ ተነባቢ ዛሬም ላገኛቸው በ...
2,ቦጋለ አበበየአዲስ አበባ ከተማ አስተዳደር ስፖርት ኮሚሽን ከኢትዮጵያ አረ...
3,ብርሃን ፈይሳአዲስ አበባ፡- የኢትዮጵያ ፕሪምየር ሊግ በሼር ካምፓኒ እንዲ...
4,ቦጋለ አበበ የኢትዮጵያ ኦሊምፒክ ኮሚቴ አርባ አምስተኛ መደበኛ ጠቅላላ ጉ...


In [ ]:
#data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}
dataset = load_dataset('csv', data_files={"train": '/content/gdrive/MyDrive/amharic/AmharicNewsDatasetTrunc.csv', "validation": '/content/gdrive/MyDrive/amharic/AmharicNewsDatasetTruncTest.csv'})

Using custom data configuration default-f45f3e5c6ad8c56e


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f45f3e5c6ad8c56e/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["article"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["article"])

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):

    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit = 3, # Only last 3 models are saved. Older ones are deleted.
    load_best_model_at_end=True
)

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic")

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 164260
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 61599


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,5.250800,5.044598
2,4.946400,4.787918
3,4.835200,4.661887


***** Running Evaluation *****
  Num examples = 41056
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-20533
Configuration saved in test-clm/checkpoint-20533/config.json
Model weights saved in test-clm/checkpoint-20533/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 41056
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-41066
Configuration saved in test-clm/checkpoint-41066/config.json
Model weights saved in test-clm/checkpoint-41066/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 41056
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-61599
Configuration saved in test-clm/checkpoint-61599/config.json
Model weights saved in test-clm/checkpoint-61599/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-clm/checkpoint-61599 (score: 4.661886692047119).


TrainOutput(global_step=61599, training_loss=5.151695551072067, metrics={'train_runtime': 15024.1589, 'train_samples_per_second': 32.799, 'train_steps_per_second': 4.1, 'total_flos': 6.735535522094592e+16, 'train_loss': 5.151695551072067, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/gdrive/MyDrive/amharic/bert_save')

Saving model checkpoint to /content/gdrive/MyDrive/amharic/bert_save
Configuration saved in /content/gdrive/MyDrive/amharic/bert_save/config.json
Model weights saved in /content/gdrive/MyDrive/amharic/bert_save/pytorch_model.bin


In [ ]:
from transformers import XLMRobertaForMaskedLM

In [ ]:
#!mv "test-clm/checkpoint-61584" "/content/gdrive/MyDrive/amharic/"